In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ELT_PySpark") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.2,com.amazonaws:aws-java-sdk-bundle:1.11.901") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin123") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()

25/05/26 13:00:33 WARN Utils: Your hostname, lenovo-PC resolves to a loopback address: 127.0.1.1; using 172.31.112.46 instead (on interface eth0)
25/05/26 13:00:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/ubuntucoy/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntucoy/.ivy2/cache
The jars for the packages stored in: /home/ubuntucoy/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-37e7bd52-109c-4173-88ec-86848e2c1f1b;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.1026 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 1283ms :: artifacts dl 35ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.1026 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.2 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	:: evicted modules:
	com.amazonaws#aws-java-sdk-bundle;1.11.901 by [com.amazonaws#aws-java-sdk-bundle;1.11.1026] in [default]
	-----------------------------------------------------

In [2]:
df = spark.read.csv("s3a://raw/erp/CUST_AZ12.csv", header=True, inferSchema=True)
df.show()

25/05/26 13:01:15 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
                                                                                

+-------------+----------+------+
|          CID|     BDATE|   GEN|
+-------------+----------+------+
|NASAW00011000|1971-10-06|  Male|
|NASAW00011001|1976-05-10|  Male|
|NASAW00011002|1971-02-09|  Male|
|NASAW00011003|1973-08-14|Female|
|NASAW00011004|1979-08-05|Female|
|NASAW00011005|1976-08-01|  Male|
|NASAW00011006|1976-12-02|Female|
|NASAW00011007|1969-11-06|  Male|
|NASAW00011008|1975-07-04|Female|
|NASAW00011009|1969-09-29|  Male|
|NASAW00011010|1969-08-05|Female|
|NASAW00011011|1969-05-03|  Male|
|NASAW00011012|1979-01-14|Female|
|NASAW00011013|1979-08-03|  Male|
|NASAW00011014|1973-11-06|Female|
|NASAW00011015|1984-08-26|Female|
|NASAW00011016|1984-10-25|  Male|
|NASAW00011017|1949-12-24|Female|
|NASAW00011018|1955-10-06|  Male|
|NASAW00011019|1983-09-04|  Male|
+-------------+----------+------+
only showing top 20 rows



In [3]:
from pyspark.sql.functions import col, when, substring, length, upper, trim, expr, current_date

# Transformasi
df_transformed = df.select(
    when(
        col("cid").like("NAS%"),
        expr("SUBSTRING(cid, 4, length(cid))")
    ).otherwise(
        col("cid")
    ).alias("cid"),       # cid: Remove 'NAS' prefix jika ada
    
    when(
        col("bdate") > current_date(),
        None
    ).otherwise(
        col("bdate")
    ).alias("bdate"),     # bdate: Set future birthdates to NULL
    
    when(
        upper(trim(col("gen"))).isin("F", "FEMALE"),
        "Female"
    ).when(
        upper(trim(col("gen"))).isin("M", "MALE"),
        "Male"
    ).otherwise(
        "n/a"
    ).alias("gen")        # gen: Normalize gender values
)
df_transformed.show()

+----------+----------+------+
|       cid|     bdate|   gen|
+----------+----------+------+
|AW00011000|1971-10-06|  Male|
|AW00011001|1976-05-10|  Male|
|AW00011002|1971-02-09|  Male|
|AW00011003|1973-08-14|Female|
|AW00011004|1979-08-05|Female|
|AW00011005|1976-08-01|  Male|
|AW00011006|1976-12-02|Female|
|AW00011007|1969-11-06|  Male|
|AW00011008|1975-07-04|Female|
|AW00011009|1969-09-29|  Male|
|AW00011010|1969-08-05|Female|
|AW00011011|1969-05-03|  Male|
|AW00011012|1979-01-14|Female|
|AW00011013|1979-08-03|  Male|
|AW00011014|1973-11-06|Female|
|AW00011015|1984-08-26|Female|
|AW00011016|1984-10-25|  Male|
|AW00011017|1949-12-24|Female|
|AW00011018|1955-10-06|  Male|
|AW00011019|1983-09-04|  Male|
+----------+----------+------+
only showing top 20 rows



In [4]:
df_transformed.write \
    .mode("overwrite") \
    .option("header", True) \
    .csv("s3a://clean/erp/CUST_AZ12_clean.csv")

25/05/26 13:01:44 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
25/05/26 13:01:44 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
                                                                                

In [5]:
df_cleaned = spark.read.option("header", True).csv("s3a://clean/erp/CUST_AZ12_clean.csv")
df_cleaned.show()

+----------+----------+------+
|       cid|     bdate|   gen|
+----------+----------+------+
|AW00011000|1971-10-06|  Male|
|AW00011001|1976-05-10|  Male|
|AW00011002|1971-02-09|  Male|
|AW00011003|1973-08-14|Female|
|AW00011004|1979-08-05|Female|
|AW00011005|1976-08-01|  Male|
|AW00011006|1976-12-02|Female|
|AW00011007|1969-11-06|  Male|
|AW00011008|1975-07-04|Female|
|AW00011009|1969-09-29|  Male|
|AW00011010|1969-08-05|Female|
|AW00011011|1969-05-03|  Male|
|AW00011012|1979-01-14|Female|
|AW00011013|1979-08-03|  Male|
|AW00011014|1973-11-06|Female|
|AW00011015|1984-08-26|Female|
|AW00011016|1984-10-25|  Male|
|AW00011017|1949-12-24|Female|
|AW00011018|1955-10-06|  Male|
|AW00011019|1983-09-04|  Male|
+----------+----------+------+
only showing top 20 rows



In [6]:
import duckdb

# Convert dari Spark ke Pandas
df_cleaned_CUST_AZ12 = df_cleaned.toPandas()

# Connect ke DuckDB
con = duckdb.connect("/mnt/d/data_engineering/duckdb/db/dev.duckdb")

# Simpan sebagai tabel DuckDB (view)
con.register("erp_CUST_AZ12_clean_view", df_cleaned_CUST_AZ12)

# Simpan sebagai tabel DuckDB (bukan view)
con.execute("CREATE OR REPLACE TABLE erp_CUST_AZ12_clean AS SELECT * FROM erp_CUST_AZ12_clean_view")

In [7]:
con.execute("SELECT * FROM erp_CUST_AZ12_clean LIMIT 10").df().style.hide(axis="index")

cid,bdate,gen
AW00011000,1971-10-06,Male
AW00011001,1976-05-10,Male
AW00011002,1971-02-09,Male
AW00011003,1973-08-14,Female
AW00011004,1979-08-05,Female
AW00011005,1976-08-01,Male
AW00011006,1976-12-02,Female
AW00011007,1969-11-06,Male
AW00011008,1975-07-04,Female
AW00011009,1969-09-29,Male


In [8]:
con.close()